## https://www.blog.pythonlibrary.org/2014/01/17/how-to-create-immutable-classes-in-python/

One of the examples I was reading about immutable classes mentioned that you could create one by replacing a class’s `__dict__` with `__slots__`. Let’s see how that looks:

In [ ]:
class Immutable(object):
    """
    An immutable class
    """
    __slots__ = ["one", "two", "three"]
    #----------------------------------------------------------------------
    def __init__(self, one, two, three):
        """Constructor"""
        super(Immutable, self).__setattr__("one", one)
        super(Immutable, self).__setattr__("two", two)
        super(Immutable, self).__setattr__("three", three)
    #----------------------------------------------------------------------
    def __setattr__(self, name, value):
        """"""
        msg = "'%s' has no attribute %s" % (self.__class__,
                                            name)
        raise AttributeError(msg)

ow we just need to create an instance of this class to see if we can monkey patch it:

In [ ]:
i = Immutable(1, 2, 3)
i.four = 4

In this case, the class does not allow us to monkey patch the instance. Instead, we receive an AttibuteError. Let’s try to change one of the attributes:

In [ ]:
i = Immutable(1, 2, 3)
i.one = 2

This is because we have overridden the `__setattr__` method. You could just override the method and not do anything at all if you wanted. This would stop the traceback from happening, but also prevent the value from being changed. If you like to be explicit with what is going on, raising an error is probably the way to go.

If you do any reading about slots, you will quickly find that using slots in this manner is discouraged. Why? Because slots were created primarily as a memory optimization (it reduces attribute access time).

Although I have never needed this, it just struck me that making an immutable object in Python could be slightly tricky. You can't just override `__setattr__`, because then you can't even set attributes in the `__init__`. Subclassing a tuple is a trick that works:

In [ ]:
class Immutable(tuple):

    def __new__(cls, a, b):
        return tuple.__new__(cls, (a, b))

    @property
    def a(self):
        return self[0]

    @property
    def b(self):
        return self[1]

    def __str__(self):
        return "<Immutable {0}, {1}>".format(self.a, self.b)

    def __setattr__(self, *ignored):
        raise NotImplementedError

    def __delattr__(self, *ignored):
        raise NotImplementedError

But then you have access to the a and b variables through self[0] and self[1], which is annoying.

Is this possible in Pure Python? If not, how would I do it with a C extension?

(Answers that work only in Python 3 are acceptable).

Update:

So subclassing tuple is the way to do it in Pure Python, which works well except for the additional possibility of accessing the data by [0], [1] etc. So, to complete this question all that is missing is howto do it "properly" in C, which I suspect would be quite simple, by just not implementing any geititem or setattribute, etc. But instead of doing it myself, I offer a bounty for that, because I'm lazy. :)

Yet another solution I just thought of: The simplest way to get the same behaviour as your original code is


In [ ]:
Immutable = collections.namedtuple("Immutable", ["a", "b"])

It does not solve the problem that attributes can be accessed via [0] etc., but at least it's considerably shorter and provides the additional advantage of being compatible with pickle and copy.

namedtuple creates a type similar to what I described in this answer, i.e. derived from tuple and using `__slots__`. It is available in Python 2.6 or above.

The easiest way to do this is using `__slots__`:

In [ ]:
class A(object):
    __slots__ = []

Instances of A are immutable now, since you can't set any attributes on them.

If you want the class instances to contain data, you can combine this with deriving from tuple:

In [ ]:
from operator import itemgetter
class Point(tuple):
    __slots__ = []
    def __new__(cls, x, y):
        return tuple.__new__(cls, (x, y))
    x = property(itemgetter(0))
    y = property(itemgetter(1))

p = Point(2, 3)
p.x

In [ ]:
p.y

Edit: If you want to get rid of indexing either, you can override `__getitem__()`:

In [ ]:
class Point(tuple):
    __slots__ = []
    def __new__(cls, x, y):
        return tuple.__new__(cls, (x, y))
    @property
    def x(self):
        return tuple.__getitem__(self, 0)
    @property
    def y(self):
        return tuple.__getitem__(self, 1)
    def __getitem__(self, item):
        raise TypeError

Note that you can't use operator.itemgetter for the properties in thise case, since this would rely on `oint.__getitem__()`instead of `uple.__getitem__()` Fuerthermore this won't prevent the use of `uple.__getitem__(p, 0)` but I can hardly imagine how this should constitute a problem.

I don't think the "right" way of creating an immutable object is writing a C extension. Python usually relies on library implementers and library users being consenting adults, and instead of really enforcing an interface, the interface should be clearly stated in the documentation. This is why I don't consider the possibility of circumventing an overridden `_setattr__()`by calling `bject.__setattr__()`a problem. If someone does this, it's on her own risk.

### https://stackoverflow.com/questions/4828080/how-to-make-an-immutable-object-in-python

You could use Cython (http://docs.cython.org/en/latest/index.html) to create an extension type for Python:

In [ ]:
cdef class Immutable:
    cdef readonly object a, b
    cdef object __weakref__ # enable weak referencing support

    def __init__(self, a, b):
        self.a, self.b = a, b

Another idea would be to completely disallow `__setattr__` and use `object.__setattr__` in the constructor:

In [ ]:
class Point(object):
    def __init__(self, x, y):
        object.__setattr__(self, "x", x)
        object.__setattr__(self, "y", y)
    def __setattr__(self, *args):
        raise TypeError
    def __delattr__(self, *args):
        raise TypeError

Of course you could use `object.__setattr__(p, "x", 3)` to modify a Point instance p, but your original implementation suffers from the same problem (try `tuple.__setattr__(i, "x", 42)` on an Immutable instance).

You can apply the same trick in your original implementation: get rid of `__getitem__()`, and use `tuple.__getitem__()` in your property functions.